# Using *Cosymlib* to analyze the symmetry properties of molecules

### Pere Alemany <br> *Institut de Química Teòrica i Computacional de la Universitat de Barcelona (IQTCUB)*

                                                                                June 2023
                                                           
This series of tutorials is meant to introduce you to using *Cosymlib*, a python library for calculating continuous shape and symmetry measures (CShMs and CSMs). In each notebook we will show different ways of using the functions in the library. 

If you run the notebook in Google Colab, you must first install the *Cosymlib*: 

In [ ]:
!pip install cosymlib

and download first a few files containing structural data that we have prepared in advance for you: 

In [ ]:
!wget  https://raw.githubusercontent.com/GrupEstructuraElectronicaSimetria/cosymlib/pere_tutorial/docs/tutorials/water.fchk
!wget  https://raw.githubusercontent.com/GrupEstructuraElectronicaSimetria/cosymlib/pere_tutorial/docs/tutorials/water.xyz
!wget  https://raw.githubusercontent.com/GrupEstructuraElectronicaSimetria/cosymlib/pere_tutorial/docs/tutorials/N2.fchk
!wget  https://raw.githubusercontent.com/GrupEstructuraElectronicaSimetria/cosymlib/pere_tutorial/docs/tutorials/CO.fchk
!wget  https://raw.githubusercontent.com/GrupEstructuraElectronicaSimetria/cosymlib/pere_tutorial/docs/tutorials/dist_water.xyz



# Using esym to compute continuous symmetry measures for the electron density


`esym` is the script used to calculate continuous symmetry measures (CSMs) with respect to  point-symmetry groups for the electron density of a molecule. For this purpose, we need to provide an electronic structure (information about the electron distribution) besides the information on the structure (position of the nuclei).  

In the first example we have previously run a quantum chemical calculation for the water molecule with an external program (Gaussian) and saved the result in the formated checkpoint format in a file called `water.fchk`. The corresponding molecular geometry is provided in the `water.xyz` file. The molecular geometry is also contained in the `water.fchk` file so that we can execute all commands just using this file. Let's look at the symmetry for the water molecule in our calculation using `gsym`:  

In [ ]:
!gsym water.fchk -pg

We can use now `esym` to compute the C2v symmetry measure for the water molecule.

In [ ]:
!esym water.fchk -m C2v

The result is that our density has the full C2v symmetry an expected (the electron density must have the same symmetry as the molecular framework. `esym` minimizes the measure with respect of the orientation of the group giving us the optimal orientation, indicating the position of the center and two axes. In the case of C2v the first axis corresponds to the C2 axis which is correctly located on the z axis. `esym` computes also the SOEVs, the symmetry operation expectation values $\text{<}{\rho}|R{\rho}{>}$ for all operations R in C2v. Since all operations in C2v are actually symmetry operations for our molecule, the result is 1.000 in all cases. `esym` returns also the self similarity for the density, that is the value of $\text{<}{\rho}|E{\rho}{>} = {<}{\rho}|{\rho}{>}$.

Computing CSMs for the density is a costly procedure since we must optimize the orientation of the group. `esym` gives us the possibility of running a cheaper calculation using two different actuation modes by including the `--mode n` tag, where n=0 is the default mode, that is, to use the actual electronic structure contained in the `.fchk` file, with n=1 we run an extended Hückel calculation to get the density and with n=2 we build a promolecular density by adding a set of non interacting spherical atomic densities.     

In [ ]:
!esym water.fchk -m C2v --mode 1

Note that by running an extended Hückel calculation we only include valence orbitals in the base, so that we only get the valence electron density. For this reason the value of the self similarity is now much lower than in the previous calculation which included also th two 1s electrons on the oxygen atom. When we run in the n=2 mode we have all electrons in the molecule, but since we have a promolecular density made from non-interacting atoms we get a slightly different value for the self similarity of the density.

In [ ]:
!esym water.fchk -m C2v --mode 2

You don't need to run a previous independent quantum chemistry program to obtain the electronic structure. If you just give a molecular structure in .xyz format to `esym` this will force an extended Hückel calculation to provide a basic electronic structure for your molecule.

In [ ]:
!esym water.xyz -m C2v

The more interesting use of `esym` is to measure the loss of symmetry in cases where we don't have the full symmetry. In the following example we compute the S(Ci) measure for the N2 and the CO molecules. In the first case the molecule is centrosymmetric while in the second case the difference in the electonegativity of the two atoms polarizes the density towards the oxygen atom.

In [ ]:
!esym N2.fchk -m Ci

In [ ]:
!esym CO.fchk -m Ci

Note that the SOEV for the inversion is now much lower, giving a large deviation for the total S(Ci) measure. This is a common situation when we use the full electron density where the mismatch between the sharp peaks in the density due to localized core electrons yield very small SOEVs. If we run an extended Hückel calculation using the n=1 mode we find that the loss of inversion symmetry is not so drastic for the valence electron density.

In [ ]:
!esym CO.fchk -m Ci --mode 1

# Using mosym to find the symmetry of molecular orbitals


`mosym` is the script used to analyze the symmetry of molecular orbitals. The SOEVs $\text{<}{\phi}|R{\phi}{>}$ for molecular orbitals take values between -1 and 1 and it is not possible to calculate a simple CSM. In this case what we compute is their character with respect of a given irreducible representation of the group. In the case that the molecule has the full symmetry of the group, we will get a 100% character for one of the irreducible representations of the group and 0% for all other. If we don't have the full symmetry we will get intermediate values between 0 and 100% for each irreducible representation indicating the mixing of different symmetries. Since in `mosym` we don't compute the optimal orientation for the group, this must be provided in the call. Let's see an example with the water molecule. To make calculations shorter we will use just extended Hückel MOs, but the calculation can be done for the full electronic structure by reading an `.fchk` file.

The `-mo` tag must be used to analyze the symmetry for all individual molecular orbitals, otherwise `mosym` will compute only the CSM for the total wave function.

In [ ]:
!mosym water.xyz -m C2v -mo -center 0.0  0.0  0.0 -axis 0.0  0.0  0.1 -axis2 1.0  0.0  0.0

Let's try now to run the same calculation for a water molecule in which we have displaced one of the H atoms. 

In [ ]:
!mosym dist_water.xyz -m C2v -mo -center 0.0  0.0  0.0 -axis 0.0  0.0  0.1 -axis2 1.0  0.0  0.0

We see that in this case the SOEVs are no longer 1.000 / 0.000 since the distortion produces different symmetry losses in each orbital. Since the distortion is small, we still have a dominating irreducible representation for each orbital and we could safely use these labels in a qualitative discussion of the symmetry for this molecule.

If we measure the C2v CSM for the whole electron density we will find just a small deviation for the density from the full C2v symmetry. We can't even see the effect on the SOEVs since we only print them with a precission of three digits. 

In [ ]:
!esym dist_water.xyz -m C2v